In [1]:
capacity = 1000
itens = [
    {
        "op": 1,
        "desc": "Ampliação da capacidade do armazém ZDP em 5%",
        "cost": 470,
        "value": 410,
    },
    {
        "op": 2,
        "desc": "Ampliação da capacidade do armazém MGL em 7%",
        "cost": 400,
        "value": 330,
    },
    {"op": 3, "desc": "Compra de empilhadeira", "cost": 170, "value": 140},
    {"op": 4, "desc": "Projeto de P&D I", "cost": 270, "value": 250},
    {"op": 5, "desc": "Projeto de P&D II", "cost": 340, "value": 320},
    {"op": 6, "desc": "Aquisição de novos equipamentos", "cost": 230, "value": 320},
    {"op": 7, "desc": "Capacitação de funcionários", "cost": 50, "value": 90},
    {
        "op": 8,
        "desc": "Ampliação da estrutura de carga rodoviária",
        "cost": 440,
        "value": 190,
    },
]

In [2]:
def gready_knapsack(itens, capacity, pre_selected=[]):
    itens.sort(key=lambda x: x["value"] / x["cost"], reverse=True)
    pre_selected_itens = [item for item in itens if item["op"] in pre_selected]
    total_value = sum([item["value"] for item in pre_selected_itens])
    total_cost = sum([item["cost"] for item in pre_selected_itens])
    selected = pre_selected

    for item in itens:
        if item["op"] in pre_selected:
            continue
        if total_cost + item["cost"] <= capacity:
            selected.append(item["op"])
            total_cost += item["cost"]
            total_value += item["value"]

    return (selected, total_value, total_cost)

In [3]:
def greedy_knapsack_with_constraints(itens, capacity):
    # if op 1 is selected, op 5 is not selected
    # if op 2 is selected, op 4 is selected

    constraints_op = [1, 5, 2, 4]

    itens_cons = [item for item in itens if item["op"] in constraints_op]
    itens_n_cons = [item for item in itens if item["op"] not in constraints_op]

    item1 = [item for item in itens_cons if item["op"] == 1][0]
    item2 = [item for item in itens_cons if item["op"] == 2][0]
    item4 = [item for item in itens_cons if item["op"] == 4][0]
    item5 = [item for item in itens_cons if item["op"] == 5][0]
    item24 = {
        "op": 24,
        "desc": "",
        "cost": item2["cost"] + item4["cost"],
        "value": item2["value"] + item4["value"],
    }

    solution = []
    # no op is selected
    solution.append(gready_knapsack(itens_n_cons, capacity))

    # op 1 is selected, 24 are not selected
    itens_to_solve = itens_n_cons + [item1]
    solution.append(gready_knapsack(itens_to_solve, capacity, [item1["op"]]))

    # op 5 is selected, 24 are not selected
    itens_to_solve = itens_n_cons + [item5]
    solution.append(gready_knapsack(itens_to_solve, capacity, [item5["op"]]))

    # op 24 is selected
    itens_to_solve = itens_n_cons + [item24]
    solution.append(gready_knapsack(itens_to_solve, capacity, [item24["op"]]))

    # op 1 and 24 are selected
    itens_to_solve = itens_n_cons + [item1, item24]
    solution.append(
        gready_knapsack(itens_to_solve, capacity, [item1["op"], item24["op"]])
    )

    # op 5 and 24 are selected
    itens_to_solve = itens_n_cons + [item5, item24]
    solution.append(
        gready_knapsack(itens_to_solve, capacity, [item5["op"], item24["op"]])
    )

    for s in solution:
        if 24 in s[0]:
            s[0].remove(24)
            s[0].append(2)
            s[0].append(4)
        print(f"Selected: {s[0]} - Total value: {s[1]} - Total cost: {s[2]}")

    # return the best solution
    return max(solution, key=lambda x: x[1])

In [4]:
def print_table(items):
    print(f"| Index | {'Descrição':50} | {'Custo':7} | {'Valor':7} |")
    print(f"| {'-'*5} | {'-'*50} | {'-'*7} | {'-'*7} |")
    for item in items:
        print(
            f'| {item["op"]:5} | {item["desc"]:50} | {item["cost"]*1000:7n} | {item["value"]*1000:7n} |'
        )
    print(f"| {'-'*5} | {'-'*50} | {'-'*7} | {'-'*7} |", end="\n\n")

In [5]:
print("Analisando os tipos de investimentos disponíveis: ")
# print md table
print_table(itens)

# print max capacity value with comma separator
print(f"Capacidade máxima de investimento: {capacity*1000:n}", end="\n\n")

Analisando os tipos de investimentos disponíveis: 
| Index | Descrição                                          | Custo   | Valor   |
| ----- | -------------------------------------------------- | ------- | ------- |
|     1 | Ampliação da capacidade do armazém ZDP em 5%       |  470000 |  410000 |
|     2 | Ampliação da capacidade do armazém MGL em 7%       |  400000 |  330000 |
|     3 | Compra de empilhadeira                             |  170000 |  140000 |
|     4 | Projeto de P&D I                                   |  270000 |  250000 |
|     5 | Projeto de P&D II                                  |  340000 |  320000 |
|     6 | Aquisição de novos equipamentos                    |  230000 |  320000 |
|     7 | Capacitação de funcionários                        |   50000 |   90000 |
|     8 | Ampliação da estrutura de carga rodoviária         |  440000 |  190000 |
| ----- | -------------------------------------------------- | ------- | ------- |

Capacidade máxima de investimento: 

In [7]:
#selected, total_value, total_cost = gready_knapsack(itens, capacity)
selected, total_value, total_cost = greedy_knapsack_with_constraints(itens, capacity)

print("\nInvestimentos selecionados:")

selected_item = [item for item in itens if item["op"] in selected]
print_table(selected_item)

# print total value with comma separator
print(f"Valor total dos investimentos selecionados: {total_value*1000:n}")
print(f"Custo total dos investimentos selecionados: {total_cost*1000:n}")

Selected: [7, 6, 5, 4, 3, 8] - Total value: 740 - Total cost: 890
Selected: [1, 7, 6, 3] - Total value: 960 - Total cost: 920
Selected: [5, 7, 6, 3] - Total value: 870 - Total cost: 790
Selected: [7, 6, 2, 4] - Total value: 990 - Total cost: 950
Selected: [1, 2, 4] - Total value: 990 - Total cost: 1140
Selected: [5, 2, 4] - Total value: 900 - Total cost: 1010

Investimentos selecionados:
| Index | Descrição                                          | Custo   | Valor   |
| ----- | -------------------------------------------------- | ------- | ------- |
|     7 | Capacitação de funcionários                        |   50000 |   90000 |
|     6 | Aquisição de novos equipamentos                    |  230000 |  320000 |
|     4 | Projeto de P&D I                                   |  270000 |  250000 |
|     2 | Ampliação da capacidade do armazém MGL em 7%       |  400000 |  330000 |
| ----- | -------------------------------------------------- | ------- | ------- |

Valor total dos investiment